# Q.6 CNN with SGD, MC Dropout, and Epistemic Uncertainty
Train a controlled convolutional neural network (CNN) on a subset of the SVHN dataset using SGD optimizer. Then, apply Monte Carlo (MC) Dropout at inference to estimate both test accuracy and epistemic uncertainty. Set random seeds to 42. Use the following configuration:
- Load SVHN and normalize pixel values to [0,1]
- Use only the first 2000 training samples and first 500 test samples
- Input shape: 32 × 32 × 3
- CNN architecture:
- Conv2D: 32 filters, 3×3 kernel, ReLU activation
- MaxPooling2D: 2×2
- Conv2D: 64 filters, 3×3 kernel, ReLU activation
- MaxPooling2D: 2×2
- Flatten
- Dense: 128 neurons, ReLU activation
- Dropout: 0.25 (keep during inference for MC Dropout)
- Output layer: 10 neurons with softmax
- Optimizer: SGD with momentum = 0.9, learning rate = 0.01
- Loss: sparse categorical crossentropy
- epochs = 15, batch size = 32
- For MC Dropout:
- Enable dropout during inference
- Average predictions over 20 stochastic forward passes
- Compute the epistemic uncertainty as the predictive variance across passes

Q6.1 Report the plain test accuracy of the CNN trained with SGD (no MC Dropout).

Q6.2 Report the MC Dropout-enhanced accuracy (averaging 20 stochastic predictions).

Q6.3 Compute the average epistemic uncertainty (mean predictive variance) across all test samples. Report it as a deterministic number rounded to 3 decimal places


# Imports, seed and data loading


In [19]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import Subset, DataLoader
import numpy as np, random

seed = 42
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

transform = transforms.Compose([transforms.ToTensor()])
train_full = datasets.SVHN('./data', split='train', download=True, transform=transform)
test_full  = datasets.SVHN('./data', split='test',  download=True, transform=transform)

train_ds = Subset(train_full, range(2000))
test_ds  = Subset(test_full,  range(500))
train_dl = DataLoader(train_ds, batch_size=32, shuffle=True)
test_dl  = DataLoader(test_ds,  batch_size=32, shuffle=False)

# CNN definition

In [20]:
# ------------------------------
# Q6.1 — Train CNN with SGD
# ------------------------------
class CNN_SVHN(nn.Module):
    def __init__(self):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, 3, padding=1), nn.ReLU(),
            nn.MaxPool2d(2,2),
            nn.Conv2d(32, 64, 3, padding=1), nn.ReLU(),
            nn.MaxPool2d(2,2)
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64*8*8, 128), nn.ReLU(),
            nn.Dropout(0.25),
            nn.Linear(128, 10)
        )

    def forward(self, x):
        return self.classifier(self.features(x))

# Model, optimizer, loss
model = CNN_SVHN().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
criterion = nn.CrossEntropyLoss()

# Training loop
epochs = 15
for epoch in range(epochs):
    model.train()
    running_loss = 0
    for x, y in train_dl:
        x, y = x.to(device), y.to(device).long()
        optimizer.zero_grad()
        out = model(x)
        loss = criterion(out, y)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(train_dl):.4f}")

# Plain test accuracy
model.eval()
correct, total = 0, 0
with torch.no_grad():
    for x, y in test_dl:
        x, y = x.to(device), y.to(device).long()
        preds = model(x).argmax(1)
        correct += (preds == y).sum().item()
        total += y.size(0)

plain_acc = 100 * correct / total
print(f"\nPlain Test Accuracy: {plain_acc:.2f}%")


Epoch 1/15, Loss: 2.2439
Epoch 2/15, Loss: 2.2325
Epoch 3/15, Loss: 2.2279
Epoch 4/15, Loss: 2.2260
Epoch 5/15, Loss: 2.2227
Epoch 6/15, Loss: 2.2199
Epoch 7/15, Loss: 2.2183
Epoch 8/15, Loss: 2.2142
Epoch 9/15, Loss: 2.2064
Epoch 10/15, Loss: 2.1948
Epoch 11/15, Loss: 2.1812
Epoch 12/15, Loss: 2.1639
Epoch 13/15, Loss: 2.1220
Epoch 14/15, Loss: 2.0645
Epoch 15/15, Loss: 1.9714

Plain Test Accuracy: 29.20%


### Q6.1 
The accuracy is low because the model doesn’t have enough data or depth to really learn the complex patterns in the SVHN images. It’s only trained on 2000 samples, and the network is pretty simple. Also, using plain SGD makes learning slower, so it doesn’t reach good performance within just 15 epochs. ( i think WIP)

### Q6.2 Report the MC Dropout-enhanced accuracy (averaging 20 stochastic predictions).



In [21]:
# ------------------------------
# Q6.2 — MC Dropout Accuracy
# ------------------------------
def mc_dropout_predictions(model, x, passes=20):
    """MC Dropout predictions: keep dropout active during inference."""
    model.train()  # ENABLE dropout
    preds = []
    for _ in range(passes):
        preds.append(torch.softmax(model(x), dim=1).unsqueeze(0))
    return torch.cat(preds, dim=0)  # [passes, batch, classes]

correct_mc, total_mc = 0, 0
with torch.no_grad():
    for x, y in test_dl:
        x, y = x.to(device), y.to(device)
        preds_mc = mc_dropout_predictions(model, x, passes=20)
        mean_preds = preds_mc.mean(0)
        final_preds = mean_preds.argmax(1)
        correct_mc += (final_preds == y).sum().item()
        total_mc += y.size(0)

mc_acc = 100 * correct_mc / total_mc
print(f"MC Dropout-enhanced Accuracy: {mc_acc:.2f}%")


MC Dropout-enhanced Accuracy: 29.20%


## Q6.3 Compute the average epistemic uncertainty (mean predictive variance) across all test samples. Report it as a deterministic number rounded to 3 decimal places

### Whats epistemic uncertainty?
Epistemic uncertainty is the uncertainty in a model’s predictions due to lack of knowledge or limited training data. It reflects how unsure the model is about its predictions and can be reduced by providing more data or improving the model. 

In practice, MC Dropout estimates it by measuring variability in predictions across multiple stochastic forward passes. High epistemic uncertainty means the model is less confident, while low uncertainty means it is more confident in its outputs.

In [22]:

import numpy as np

epistemic_vars = []

with torch.no_grad():
    for x, _ in test_dl:
        x = x.to(device)
        preds_mc = mc_dropout_predictions(model, x, passes=20)  # [20, batch, 10]
        # variance per sample over stochastic passes
        var_preds = preds_mc.var(0).mean(1)  # mean over classes
        epistemic_vars.extend(var_preds.cpu().numpy())

avg_epistemic_uncertainty = np.mean(epistemic_vars)
print(f"Average Epistemic Uncertainty: {avg_epistemic_uncertainty:.3f}")


Average Epistemic Uncertainty: 0.000


Gonna be honest. i dont how why its rolling with a 0% uncertainty. This still presists when changing the pass amount. Might have to fix this tomorrow :/ 